In [1]:
import numpy as np
from cellpose import models, core, io
from pathlib import Path
from pathlib import Path
import napari
import apoc
from tqdm import tqdm
import pyclesperanto_prototype as cle 
from tifffile import imwrite, imread
from utils import list_images, read_image

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


2025-11-01 14:18:00,247 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-11-01 14:18:00,247 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-11-01 14:18:00,320 [INFO] ** TORCH CUDA version installed and working. **
2025-11-01 14:18:00,320 [INFO] ** TORCH CUDA version installed and working. **
2025-11-01 14:18:00,320 [INFO] >>>> using GPU (CUDA)
2025-11-01 14:18:01,216 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam


In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\25.09.19\Plate 1 (occludin + claudin-2)")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E2_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E3_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E4_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E5_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E6_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E7_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie

In [3]:
viewer = napari.Viewer(ndisplay=2)

In [4]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id
well_id = filename.split("_")[0]

# Open one of the multipositions in the img file
img = img[2]

viewer.add_image(img)


Image analyzed: E2_20x


<Image layer 'img' at 0x1a58cad1f30>

In [5]:
img.shape

(15, 5, 1360, 1360)

In [6]:
img = img.transpose(1, 0, 2, 3)
img.shape

(5, 15, 1360, 1360)

In [7]:
# Cellmask and nuclei are in position 2 and 3 respectively

# if you want to combine two stains to create your "cytoplasm" channel
# in this example indices 0, 1 and 2 (1st, 2nd and 3rd) have cellular stains
# and nuclei are in index 3 (4th channel)

img_cp = np.stack((img[[0,1,2]].sum(axis=0), img[3]), axis=0)

img_cp.shape

(2, 15, 1360, 1360)

In [8]:
viewer.add_image(img_cp)

<Image layer 'img_cp' at 0x1a583670be0>

In [9]:
labels, flows, styles = model.eval(img_cp, channel_axis=0, z_axis=1, do_3D=True, niter=None) # need to check the arguments
viewer.add_labels(labels)

2025-11-01 14:29:00,104 [INFO] running YX: 15 planes of size (1360, 1360)
2025-11-01 14:29:13,489 [INFO] 100%|##########| 15/15 [00:13<00:00,  1.12it/s]
2025-11-01 14:29:13,628 [INFO] running ZY: 1360 planes of size (15, 1360)
2025-11-01 14:30:12,330 [INFO] 35%|###4      | 474/1360 [00:58<01:49,  8.07it/s]


KeyboardInterrupt: 

In [10]:
maxproj = np.max(img_cp, axis=1)

In [11]:
viewer.add_image(maxproj)

<Image layer 'maxproj' at 0x1a582861420>

In [12]:
maxproj.shape

(2, 1360, 1360)

In [13]:
labels, flows, styles = model.eval(maxproj[1], do_3D=False, niter=None) # simple nuclei segmentation
viewer.add_labels(labels)

<Labels layer 'labels' at 0x1a591c12ce0>

In [ ]:
cle.voronoi_otsu_labeling()

In [14]:
import nd2

def extract_scaling_metadata(filepath):

    with nd2.ND2File(filepath) as nd2_data:
        # Get the first channel's volume metadata
        first_channel = nd2_data.metadata.channels[0]
        voxel_size = first_channel.volume.axesCalibration  # X, Y, Z calibration

        # Extract pixel sizes
        pixel_size_x, pixel_size_y, voxel_size_z = voxel_size

        print(f"Pixel size: {pixel_size_x:.3f} µm x {pixel_size_y:.3f} µm")
        print(f"Voxel (Z-step) size: {voxel_size_z:.3f} µm")

    return pixel_size_x, pixel_size_y, voxel_size_z

In [15]:
img_cp.shape

(2, 15, 1360, 1360)

In [16]:
# Extract x,y,z scaling from .czi file metadata in order to make data isotropic
scaling_x_um, scaling_y_um, scaling_z_um = extract_scaling_metadata(image)

# Adjust so voxel size_x and size_y so they are equal to 1 to avoid compression upon rescaling
multiplier = 1 / scaling_x_um

scaling_x_um = scaling_x_um * multiplier
scaling_y_um = scaling_y_um * multiplier
scaling_z_um = scaling_z_um * multiplier

resampled = cle.scale(img_cp[1], factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)

Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 6.631 µm


In [17]:
viewer.add_image(resampled)

<Image layer 'resampled' at 0x1a592039510>

2025-11-01 14:35:23,035 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [ ]:
cle_labels = cle.voronoi_otsu_labeling(resampled)
viewer.add_labels(cle_labels)

2025-11-01 14:37:53,370 [INFO] build program: kernel 'gaussian_blur_separable_3d' was part of a lengthy uncached source build (assuming cached by ICD) (0.28 s)


<Labels layer 'cle_labels' at 0x1a82e500d00>

In [21]:
def simulate_cytoplasm(nuclei_labels, dilation_radius=2, erosion_radius=0):

    if erosion_radius >= 1:

        # Erode nuclei_labels to maintain a closed cytoplasmic region when labels are touching (if needed)
        eroded_nuclei_labels = cle.erode_labels(nuclei_labels, radius=erosion_radius)
        eroded_nuclei_labels = cle.pull(eroded_nuclei_labels)
        nuclei_labels = eroded_nuclei_labels

    # Dilate nuclei labels to simulate the surrounding cytoplasm
    cyto_nuclei_labels = cle.dilate_labels(nuclei_labels, radius=dilation_radius)
    cytoplasm = cle.pull(cyto_nuclei_labels)

    # Create a binary mask of the nuclei
    nuclei_mask = nuclei_labels > 0

    # Set the corresponding values in the cyto_nuclei_labels array to zero
    cytoplasm[nuclei_mask] = 0

    return cytoplasm

In [23]:
cytoplasm = simulate_cytoplasm(labels, dilation_radius=3, erosion_radius=1)
viewer.add_labels(cytoplasm)

<Labels layer 'cytoplasm [1]' at 0x1a597eea1a0>